<a href="https://colab.research.google.com/github/kiratkJaura/Final-Project-02/blob/main/Final_Project_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Final Project 02

In [ ]:
# AER 850  Project 2
# Author: Kirat Kaur
# Student No: 501125524

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Data Processing

image_size = (500, 500)
batch_size = 32

# Directories for training, validation, and testing data

train_dir = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/valid'
test_dir = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/test'

# Image data generators with augmentation for training and simple scaling for validation/testing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Creating generators
train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')

# Retrieving class names based on directory structure
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

# Step 2: Neural Network Architecture Design
model = models.Sequential()

# First Convolutional layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Second Convolutional layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Third Convolutional layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flattening and adding Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))                    # Dropout to reduce overfitting
model.add(layers.Dense(3, activation='softmax'))  # Output layer for 3 classes

# Step 3: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Model Training and Evaluation with Early Stopping

# Training the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  ## early stopping

history = model.fit(
   train_generator,
   steps_per_epoch=train_generator.samples // batch_size,
   epochs=15,
   validation_data=val_generator,
   validation_steps=val_generator.samples // batch_size,
   callbacks=[early_stopping]
)


# Plots for training & validation accuracy, and loss

# Plotting accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

# Plotting loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Step 5: Model Testing

# Function to preprocess test image
def preprocess_test_image(img_path):
    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

# Predicting on a single test images for crack, missing-head, and paint-off

# Crack single image test
test_img_path = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/test/crack/test_crack.jpg'
test_img = preprocess_test_image(test_img_path)
prediction = model.predict(test_img)

# Missing-head single image test
test_img_path = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/test/missing-head/test_missinghead.jpg'
test_img = preprocess_test_image(test_img_path)
prediction = model.predict(test_img)

# Paint-off single image test
test_img_path = '/content/drive/MyDrive/Colab Notebooks/Project 02; AER 850/Data/test/paint-off/test_paintoff.jpg'
test_img = preprocess_test_image(test_img_path)
prediction = model.predict(test_img)

# Finding the predicted class
predicted_class = np.argmax(prediction)
class_names = ['Crack', 'Missing Head', 'Paint Off']
print(f"The model predicts this image as: {class_names[predicted_class]}")


Found 1942 images belonging to 3 classes.
Found 431 images belonging to 3 classes.
Class names: ['crack', 'missing-head', 'paint-off']


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 1471s 23s/step - accuracy: 0.3863 - loss: 3.8583 - val_accuracy: 0.4231 - val_loss: 1.0226
Epoch 2/15
 1/60 ━━━━━━━━━━━━━━━━━━━━ 20:56 21s/step - accuracy: 0.4062 - loss: 1.0033

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.4062 - loss: 1.0033 - val_accuracy: 0.3333 - val_loss: 0.9874
Epoch 3/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 1317s 21s/step - accuracy: 0.5008 - loss: 0.9885 - val_accuracy: 0.5216 - val_loss: 0.8566
Epoch 4/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 36ms/step - accuracy: 0.5938 - loss: 0.8709 - val_accuracy: 0.5333 - val_loss: 0.8755
Epoch 5/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 1272s 21s/step - accuracy: 0.5986 - loss: 0.8519 - val_accuracy: 0.5625 - val_loss: 0.8237
Epoch 6/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.6250 - loss: 0.7601 - val_accuracy: 0.6000 - val_loss: 0.6980
Epoch 7/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 1320s 21s/step - accuracy: 0.5910 - loss: 0.8423 - val_accuracy: 0.6130 - val_loss: 0.7649
Epoch 8/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.6875 - loss: 0.5788 - val_accuracy: 0.6000 - val_loss: 0.7152
Epoch 9/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 1265s 21s/step - accuracy: 0.5967 - loss: 0.8257 - val_accuracy: 0.5865 - 